## Problem Description:

● Certain organization's advancements are shared across different channels including
email, notices and so forth.

● Some of these campaigns incorporate coupon limits that are offered for a particular item
or scope of items.

● The retailer might want the capacity to anticipate whether clients recover the coupons
got across channels, which will help the retailer's advertising group to precisely plan
coupon builds, and grow progressively exact and focused on promoting techniques.

● The information accessible right now the beneath data, including the subtleties of an
example of: User demographics, Campaign and coupon details, Product details,
Previous transactions.

● In light of past exchange and execution information from the last 18 campaigns, the
candidates were suggested to anticipate the **probability of a coupon being reclaimed**
on the proper validation cut that should be made

## Approach

**Steps 1: Data Loading and Exploring**
- Loading Libraries and Data files.

**Step 2: Data Cleaning and Formatting**
- Data Exploring (info, sort_values, nunique, isna, ..)
- Cleaning Data and imputing nulls.
- Handling negatives and categorical values.

**Step 3: Data Merging and Wrangling**
- Merging dataframes on common keys.
- Checking and dropping duplicates.

**Step 4: Data Analysis**
- Analysing merged tables and dropping non-significant columns.
- Relation Analysis between campaign_id, coupon_id and customer_id
- Merging all the given data together.
- Analying nulls on merged dataframe (Huge nulls in alot of columns)
- Feature engineering on merged dataframe. (BinaryEncoder, dummy_variables, ..)
- Default model on the final dataset after merging.
- Analyzing performance and finding better approach.
- 2nd Approach = Went ahead with merged train and campaign_data only.

**Step 5: Feature Engineering**
- Dropping insignificant columns.
- Converting campaign_id, campaign_type, coupon_id, customer_id into category.
- BinaryEncoder to get less features but performance was worse.
- Creating dummy features instead(Huge number of features)
- Checking performance on a default logistic model (much better performance)
- Downcast all the features to save memory.

**Step 6: Validation Strategy**
- Train-Test split
- Checking cross validation score on 8 default models.
(LogisticRegression, Bagging, DecisionTree, Random_Forest, GBM, XGBoost, Art.Neural_Net, CatBoost)
- Evaluation matrix - roc_auc_score
- Dropping models with less score & Hypertuning models with high score
- Checking cv roc_auc score on hypertuned models and finding best estimators and best score.
- Checking validation set performance with best estimators on best models.
- Checking validation performance of Ensemble Models

**Step 7: Select Final Model**
- Ensemble 1: roc_auc:
- Ensemble 2: roc_auc:
- Ensemble 3: roc_auc:

**Final Model = Ensemble 2**

## Load Libraries

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')
import seaborn as sns
pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Importing modelling libraries
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler  
from sklearn.metrics import accuracy_score,roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,VotingClassifier,BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.calibration import CalibratedClassifierCV
pd.options.display.float_format = "{:,.2f}".format

# to ignore warnings:
import sys
if not sys.warnoptions:
    import os, warnings
    warnings.simplefilter("ignore") 
    os.environ["PYTHONWARNINGS"] = "ignore" 
    
import time
from contextlib import contextmanager
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} done in {:.0f}s".format(title, time.time() - t0))


## Load Datasets

In [2]:
train = pd.read_csv('train.csv')
campaign_data = pd.read_csv('campaign_data.csv')
item_data = pd.read_csv('item_data.csv')
coupon_item = pd.read_csv('coupon_item_mapping.csv')
customer_demo = pd.read_csv('customer_demographics.csv')
customer_trans = pd.read_csv('customer_transaction_data.csv')
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


## Data Cleaning, Merging and Wrangling.

Let's start with Customer demographics data.

### Customer Demographics

In [3]:
customer_demo.sort_values(by='customer_id')[:10]

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5
5,11,70+,Single,0,2,NaN,1
6,12,46-55,Married,0,2,NaN,7
7,13,36-45,Single,0,1,NaN,2
8,14,26-35,Married,1,2,NaN,6
9,15,46-55,Married,0,2,NaN,6


In [4]:
customer_demo.isna().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [5]:
customer_demo['family_size'] = customer_demo['family_size'].str.replace('+','')

In [6]:
#Fill nans
customer_demo.no_of_children.fillna('0',inplace=True)
for i in range(len(customer_demo)):
    if int(customer_demo.family_size[i]) >= 2:
        customer_demo.marital_status[i] = 'Married'
    else:
        customer_demo.marital_status[i] = 'Single'

In [7]:
customer_demo.sample(5)

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
618,1295,36-45,Married,0,2,0,4
148,293,56-70,Single,0,1,0,4
637,1328,56-70,Married,0,2,0,10
181,368,36-45,Single,0,1,0,5
518,1068,46-55,Married,0,2,0,2


Now let's check customer transaction.

### Customer Transactions

In [8]:
customer_trans.sample(5)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
566112,2012-10-02,1121,32650,1,124.31,0.00,0.00
480751,2012-09-02,51,12679,1,117.19,-24.93,0.00
697365,2012-11-17,470,24377,1,106.50,0.00,0.00
412329,2012-08-09,722,18325,1,56.64,-14.25,0.00
720774,2012-11-25,1161,19021,1,59.49,-32.77,0.00


In [9]:
#to handle negative values
customer_trans.other_discount = customer_trans.other_discount.abs()  

We'll start by merging customer Demographics and Customer Transactions.

In [10]:
customer_demo.info() , customer_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     760 non-null    int64 
 1   age_range       760 non-null    object
 2   marital_status  760 non-null    object
 3   rented          760 non-null    int64 
 4   family_size     760 non-null    object
 5   no_of_children  760 non-null    object
 6   income_bracket  760 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 41.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324566 entries, 0 to 1324565
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   date             1324566 non-null  object 
 1   customer_id      1324566 non-null  int64  
 2   item_id          1324566 non-null  int64  
 3   quantity         1324566 non-null  int64  
 4   selling_price    1324566 non-null  float64
 

(None, None)

Data for 760 customers but transactions above 13 lakhs.

In [11]:
customer_demo.nunique(), customer_trans.nunique()

(customer_id       760
 age_range           6
 marital_status      2
 rented              2
 family_size         5
 no_of_children      4
 income_bracket     12
 dtype: int64,
 date                 549
 customer_id         1582
 item_id            74063
 quantity            9252
 selling_price       4923
 other_discount      1418
 coupon_discount      232
 dtype: int64)

So we have customer demographics for 760 customers while we have transactions data for 1582 customers.
So let's merge customer demograhics data with customer transactions.

### Customer_data = Merged(Customer Transaction + Customer Demographics)

In [12]:
customer_data = customer_trans.merge(customer_demo, on='customer_id', how='left')

In [13]:
customer_data.sample(5)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,age_range,marital_status,rented,family_size,no_of_children,income_bracket
904424,2013-01-30,785,5985,1,35.62,0.00,0.00,36-45,Married,0.00,3,1,5.00
611737,2012-10-18,1312,64150,2,71.24,20.66,0.00,NaN,NaN,nan,NaN,NaN,nan
1021554,2013-03-13,1558,49660,15,"1,031.20",299.21,-534.30,36-45,Married,0.00,3,1,6.00
101207,2012-04-09,231,50305,1,224.05,24.93,0.00,46-55,Single,0.00,1,0,5.00
357880,2012-07-19,802,13505,1,44.52,26.36,0.00,46-55,Single,0.00,1,0,2.00


In [14]:
customer_data.fillna('NA', inplace=True)

In [15]:
#Check and remove duplicates
customer_data[customer_data.duplicated(keep=False)][:10]

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,age_range,marital_status,rented,family_size,no_of_children,income_bracket
5463,2012-01-19,801,32650,3,213.72,159.22,0.00,26-35,Single,0.00,1,0,2.00
5465,2012-01-19,801,32650,3,213.72,159.22,0.00,26-35,Single,0.00,1,0,2.00
5491,2012-01-19,814,25251,2,163.14,0.00,0.00,NA,NA,NA,NA,NA,NA
5493,2012-01-19,814,57670,2,21.37,0.00,0.00,NA,NA,NA,NA,NA,NA
5497,2012-01-19,814,25251,2,163.14,0.00,0.00,NA,NA,NA,NA,NA,NA
5502,2012-01-19,814,57670,2,21.37,0.00,0.00,NA,NA,NA,NA,NA,NA
5508,2012-01-19,433,34474,1,89.05,17.45,0.00,NA,NA,NA,NA,NA,NA
5514,2012-01-19,433,34474,1,89.05,17.45,0.00,NA,NA,NA,NA,NA,NA
6027,2012-01-20,1140,28197,1,106.50,0.00,0.00,NA,NA,NA,NA,NA,NA
6038,2012-01-20,1140,28197,1,106.50,0.00,0.00,NA,NA,NA,NA,NA,NA


In [16]:
customer_data.drop_duplicates(keep=False, inplace=True)

Now we have a customer_data dataframe where we have customer transactions along with the available customer demographics.


### Coupon Item Mapping and Item Data

In [17]:
coupon_item.info(), item_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92663 entries, 0 to 92662
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   coupon_id  92663 non-null  int64
 1   item_id    92663 non-null  int64
dtypes: int64(2)
memory usage: 1.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74066 entries, 0 to 74065
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   item_id     74066 non-null  int64 
 1   brand       74066 non-null  int64 
 2   brand_type  74066 non-null  object
 3   category    74066 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


(None, None)

In [18]:
coupon_item.nunique(), item_data.nunique()

(coupon_id     1116
 item_id      36289
 dtype: int64,
 item_id       74066
 brand          5528
 brand_type        2
 category         19
 dtype: int64)

In [19]:
item_data.sort_values(by='item_id')[:10]

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery
5,6,56,Local,Grocery
6,7,56,Local,Pharmaceutical
7,8,56,Local,Bakery
8,9,11,Local,Grocery
9,10,56,Local,Grocery


In [20]:
coupon_item.sort_values(by=['item_id','coupon_id'])[:10]

,coupon_id,item_id
41301,22,1
80209,31,1
31530,82,4
18860,166,4
30395,327,4
54703,23,7
77779,32,7
40075,22,10
74574,31,10
30070,317,10


This shows that on merging, one item can have multiple coupon codes.

In [21]:
coupon_item.sort_values(by=['coupon_id','item_id'])[:10]

,coupon_id,item_id
29187,1,4007
29956,1,4140
28628,1,17091
30043,1,24181
29732,1,43951
31203,1,43986
31154,1,44076
28629,1,44112
30837,1,44491
29730,1,44530


### Item Coupon = Merged(Item Data + Coupon Item Mapping)

In [22]:
item_coupon = item_data.merge(coupon_item, on='item_id', how='left')

In [23]:
item_coupon.sort_values(by='item_id')[:10]

,item_id,brand,brand_type,category,coupon_id
0,1,1,Established,Grocery,22.00
1,1,1,Established,Grocery,31.00
2,2,1,Established,Miscellaneous,nan
3,3,56,Local,Bakery,nan
4,4,56,Local,Grocery,166.00
5,4,56,Local,Grocery,327.00
6,4,56,Local,Grocery,82.00
7,5,56,Local,Grocery,nan
8,6,56,Local,Grocery,nan
9,7,56,Local,Pharmaceutical,23.00


There are many items which don't have any coupon id, let's substitute the nulls in coupon_id with zeros.

In [24]:
item_coupon.coupon_id.fillna('0', inplace=True)

Now we can merge item_coupon with customer_data on item_id. 

### Customer Item = Merged( Customer Data + Item Coupon )

In [77]:
customer_item = customer_data.merge(item_coupon, on='item_id', how='left')

In [78]:
customer_item.sample(5)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,age_range,marital_status,rented,family_size,no_of_children,income_bracket,brand,brand_type,category,coupon_id
363830,2012-05-13,706,9136,1,320.22,0.00,0.00,NA,NA,NA,NA,NA,NA,212,Established,Grocery,0
913628,2012-08-25,867,48923,1,142.12,0.00,0.00,46-55,Married,0.00,2,0,5.00,1488,Established,Grocery,0
677216,2012-07-13,254,30681,1,59.13,47.37,0.00,NA,NA,NA,NA,NA,NA,278,Established,Grocery,529.00
2136267,2013-03-30,186,13774,1,64.12,3.56,0.00,26-35,Married,0.00,2,0,4.00,56,Local,Grocery,9.00
2000164,2013-03-05,266,36547,1,89.05,17.45,0.00,56-70,Married,0.00,5,3+,4.00,79,Established,Packaged Meat,8.00


In [79]:
customer_item.coupon_id = customer_item.coupon_id.astype(int)

In [80]:
customer_item.isna().sum().sort_values(ascending=False)[:3]

coupon_id      0
age_range      0
customer_id    0
dtype: int64

In [81]:
customer_item[customer_item.duplicated(keep=False)][:10] #No duplicates

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,age_range,marital_status,rented,family_size,no_of_children,income_bracket,brand,brand_type,category,coupon_id


In [82]:
customer_item.sort_values(by=['customer_id','coupon_id','item_id'])[110:120]

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,age_range,marital_status,rented,family_size,no_of_children,income_bracket,brand,brand_type,category,coupon_id
2035804,2013-03-12,1,10446,1,95.82,0.00,0.00,70+,Married,0.00,2,0,4.00,522,Established,"Dairy, Juices & Snacks",0
2127158,2013-03-28,1,10446,1,95.82,0.00,0.00,70+,Married,0.00,2,0,4.00,522,Established,"Dairy, Juices & Snacks",0
2318732,2013-05-03,1,10446,1,95.82,0.00,0.00,70+,Married,0.00,2,0,4.00,522,Established,"Dairy, Juices & Snacks",0
2418886,2013-05-21,1,10446,1,95.82,0.00,0.00,70+,Married,0.00,2,0,4.00,522,Established,"Dairy, Juices & Snacks",0
2472079,2013-05-31,1,10446,1,95.82,0.00,0.00,70+,Married,0.00,2,0,4.00,522,Established,"Dairy, Juices & Snacks",0
179989,2012-04-04,1,10534,1,89.05,10.33,0.00,70+,Married,0.00,2,0,4.00,49,Established,Grocery,0
965171,2012-09-03,1,10534,1,79.79,26.71,0.00,70+,Married,0.00,2,0,4.00,49,Established,Grocery,0
1836248,2013-02-04,1,10534,1,89.05,35.26,0.00,70+,Married,0.00,2,0,4.00,49,Established,Grocery,0
528587,2012-06-14,1,10637,1,106.50,0.00,0.00,70+,Married,0.00,2,0,4.00,619,Established,Grocery,0
1616617,2012-12-27,1,10699,1,106.50,0.00,0.00,70+,Married,0.00,2,0,4.00,1074,Established,Bakery,0


If the customer is making same transactions on different dates, this doesn't provide us any knowledge about the redemption status as all values are similar, so we can go ahead and drop the date column and remove the duplicate rows.

**We can create a lot of features with the date column- month/week/year/dayofweek/is_weekend/.... but let's merge the datasets before and after we have our final dataset, will do feature engineering then.**

In [32]:
customer_item.shape

(2644439, 16)

### Campaign Data and Train Data

In [33]:
campaign_data.sort_values(by='campaign_id')[:10] #unique campaigns

,campaign_id,campaign_type,start_date,end_date
21,1,Y,12/12/12,18/01/13
22,2,Y,17/12/12,18/01/13
18,3,Y,22/12/12,16/02/13
20,4,Y,07/01/13,08/02/13
19,5,Y,12/01/13,15/02/13
17,6,Y,28/01/13,01/03/13
16,7,Y,02/02/13,08/03/13
15,8,X,16/02/13,05/04/13
14,9,Y,11/03/13,12/04/13
13,10,Y,08/04/13,10/05/13


In [34]:
train.sort_values(by=['campaign_id','coupon_id','customer_id'])[:10]

,id,campaign_id,coupon_id,customer_id,redemption_status
56024,92117,1,312,46,0
60615,99597,1,312,63,0
3361,5611,1,312,72,0
56938,93608,1,312,188,0
70177,115275,1,312,340,0
61966,101861,1,312,344,0
23610,38711,1,312,362,0
66597,109327,1,312,401,0
3156,5245,1,312,595,0
56333,92620,1,312,787,0


In [35]:
train[train.duplicated(keep=False)][:10]

,id,campaign_id,coupon_id,customer_id,redemption_status


**Train tells us that redemption status remains same, doesn't matter how many transactions are done with the same coupon_id and customer_id.**

### Campaign Train = Merged( Train + Campaign Data )

In [63]:
campaign_train = train.merge(campaign_data, on='campaign_id', how = 'left')

In [64]:
campaign_train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date
0,1,13,27,1053,0,X,19/05/13,05/07/13
1,2,13,116,48,0,X,19/05/13,05/07/13
2,6,9,635,205,0,Y,11/03/13,12/04/13
3,7,13,644,1050,0,X,19/05/13,05/07/13
4,9,8,1017,1489,0,X,16/02/13,05/04/13


Here start_date and end_date doesn't ptovide any value to our analysis.

In [65]:
campaign_train.drop(['start_date','end_date'],axis=1,inplace=True)

In [66]:
campaign_train.sort_values(ascending=True, by=['customer_id','coupon_id','campaign_id'])[20:30]

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type
29000,47528,8,276,1,0,X
74786,122796,13,281,1,0,X
50024,82060,13,293,1,0,X
51584,84658,29,416,1,0,Y
71345,117187,29,418,1,0,Y
13683,22582,13,422,1,0,X
3062,5085,8,424,1,0,X
11527,19022,29,443,1,0,Y
14878,24525,29,445,1,0,Y
67132,110196,29,448,1,0,Y


In [67]:
campaign_train[campaign_train.duplicated(keep=False)][:10]

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type


In Campaign train, we have redemption status for customer_id and coupon_ids with no duplicates, but in the customer_item dataframe there might be multiple items with same customer_id and coupon_id..so we can just drop the duplicates as the redemption status with multiple items remains uneffected.

### Train = Merged( Campaign Train + Customer Item )

In [83]:
#Drop duplicates so the train table maintains it's shape.
train1 = campaign_train.merge(customer_item.drop_duplicates(subset=['customer_id','coupon_id']), how='left')


In [84]:
train1.shape

(78369, 20)

Our final merged data has exactly the same number of rows as our initial train data.

In [85]:
train1.sample(5)

,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,date,item_id,quantity,selling_price,other_discount,coupon_discount,age_range,marital_status,rented,family_size,no_of_children,income_bracket,brand,brand_type,category
67350,26,710,1547,0,0,2012-01-28,"51,825.00",1.00,106.50,42.74,0.00,46-55,Married,0.00,2,0,5.00,717.00,Established,Grocery
70229,10,732,1182,0,1,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN
43858,11,658,1115,0,1,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN
67810,8,52,1080,0,0,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN
6068,8,57,1238,0,0,NaN,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN


In [86]:
train1.isna().sum()

campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
campaign_type            0
date                 64586
item_id              64586
quantity             64586
selling_price        64586
other_discount       64586
coupon_discount      64586
age_range            64586
marital_status       64586
rented               64586
family_size          64586
no_of_children       64586
income_bracket       64586
brand                64586
brand_type           64586
category             64586
dtype: int64

Model performance after feature engineering on this train1 was poor, default LogisticRegression gave an roc_auc score of only 0.78

Majority of the columns have null values. This means that for unique customer_id and coupon_id, majority of the customer transaction data is null.
This shows that the items and customers doesn't provide much value for our Analysis.So we should rather go ahead with only campaign_train data.

In [68]:
#Let's do feature engineering on campaign_train.
campaign_train.sample(5)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type
7409,12289,8,920,1084,0,X
9013,14884,13,22,235,0,X
76359,125336,8,754,992,0,X
13499,22279,4,895,924,0,Y
67780,111285,2,1023,833,0,Y


In [69]:
campaign_train.drop('id',axis=1,inplace=True)
campaign_train.shape

(78369, 5)

In [70]:
campaign_train.nunique(), campaign_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78369 entries, 0 to 78368
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   campaign_id        78369 non-null  int64 
 1   coupon_id          78369 non-null  int64 
 2   customer_id        78369 non-null  int64 
 3   redemption_status  78369 non-null  int64 
 4   campaign_type      78369 non-null  object
dtypes: int64(4), object(1)
memory usage: 3.6+ MB


(campaign_id            18
 coupon_id             866
 customer_id          1428
 redemption_status       2
 campaign_type           2
 dtype: int64,
 None)

In [71]:
campaign_train.campaign_type.replace(to_replace=['X', 'Y'], value=[0, 1], inplace=True)
campaign_train.campaign_id = campaign_train.campaign_id.astype('category')
campaign_train.coupon_id = campaign_train.coupon_id.astype('category')
campaign_train.customer_id = campaign_train.customer_id.astype('category')

approach2 = pd.get_dummies(campaign_train)
approach2.head()

redemption_status  campaign_type  campaign_id_1  campaign_id_2  \
0                  0              0              0              0   
1                  0              0              0              0   
2                  0              1              0              0   
3                  0              0              0              0   
4                  0              0              0              0   

   campaign_id_3  campaign_id_4  campaign_id_5  campaign_id_6  campaign_id_7  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   campaign_id_8  campaign_id_9  campaign_id_10  campaign_id_11  \
0              0              0               0               0   
1              0              0               0               0   
2              0              1               0               0   
3              0              0               0               0   
4              1              0               0               0   

   campaign_id_12  campaign_id_13  campaign_id_26  campaign_id_27  \
0               0               1               0               0   
1               0               1               0               0   
2               0               0               0               0   
3               0               1               0               0   
4               0               0               0               0   

   campaign_id_28  campaign_id_29  campaign_id_30  coupon_id_1  coupon_id_2  \
0               0               0               0            0            0   
1               0               0               0            0            0   
2               0               0               0            0            0   
3               0               0               0            0            0   
4               0               0               0            0            0   

   coupon_id_3  coupon_id_4  coupon_id_5  coupon_id_6  coupon_id_7  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   coupon_id_8  coupon_id_9  coupon_id_10  coupon_id_11  coupon_id_12  \
0            0            0             0             0             0   
1            0            0             0             0             0   
2            0            0             0             0             0   
3            0            0             0             0             0   
4            0            0             0             0             0   

   coupon_id_13  coupon_id_14  coupon_id_15  coupon_id_16  coupon_id_17  \
0             0             0             0             0             0   
1             0             0             0             0             0   
2             0             0             0             0             0   
3             0             0             0             0             0   
4             0             0             0             0             0   

   coupon_id_18  coupon_id_19  coupon_id_20  coupon_id_21  coupon_id_22  \
0             0             0             0             0             0   
1             0             0             0             0             0   
2             0             0             0             0             0   
3             0             0             0             0             0   
4             0             0             0             0             0   

   coupon_id_23  coupon_id_24  coupon_id_25  coupon_

In [72]:
approach2.shape

(78369, 2314)

We now have 2314 features in our train data. Let's down cast it and check default model performance.
### Downcast data types

In [73]:
#downcast datatypes to save memory
fcols = approach2.select_dtypes('float').columns
icols = approach2.select_dtypes('integer').columns

approach2[fcols] = approach2[fcols].apply(pd.to_numeric, downcast='float')
approach2[icols] = approach2[icols].apply(pd.to_numeric, downcast='integer')

## Data Modeling

### Splitting the data as train and validation data

In [74]:
x = approach2.drop('redemption_status',axis=1)
y = approach2['redemption_status']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.20, random_state = 99)

### Performance on Default Model

In [75]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

pred = lr.predict_proba(x_val)[:,1]
print(roc_auc_score(y_val, pred))

0.9369623693540647


The performance of default logistic regression model is amazing on Validation set.

### Cross Validation Accuracy on Default Models

In [312]:
# We will check scores for these 8 default models and then we will check their best scores after tuning. 
r = 22
models = [LogisticRegression(random_state=r),BaggingClassifier(random_state=r),DecisionTreeClassifier(random_state=r),
          RandomForestClassifier(random_state=r), GradientBoostingClassifier(random_state=r),
          XGBClassifier(random_state=r), MLPClassifier(random_state=r),
          CatBoostClassifier(random_state=r,verbose = False)]

names = ["LogisticRegression","Bagging","DecisionTree","Random_Forest","GBM",
         "XGBoost","Art.Neural_Network","CatBoost"]

In [313]:
results = []
print('5 fold Cross validation accuracy and std of the default models for the train data:', end = "\n\n")
for name, model in zip(names, models):
    kfold = KFold(n_splits=5, random_state=1001)
    cv_results = cross_val_score(model, x, y, cv = kfold, scoring = "roc_auc")
    results.append(cv_results)
    print("{}: {} ({})".format(name, "%.3f" % cv_results.mean() ,"%.3f" %  cv_results.std()))

5 fold Cross validation accuracy and std of the default models for the train data:

LogisticRegression: 0.905 (0.016)
Bagging: 0.540 (0.011)
DecisionTree: 0.531 (0.006)
Random_Forest: 0.859 (0.028)
GBM: 0.788 (0.025)
[09:48:50] WARNING: /Users/adamyanayyar/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:55:57] WARNING: /Users/adamyanayyar/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:03:07] WARNING: /Users/adamyanayyar/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learne

Suprisingly, the boosting models didn't give us high roc_auc score. So for our model, we will hypertune Logistic Regression, Random Forest and Art.Neural Network.


### Model tuning with Cross Validation

In [58]:
r= 2033
final_models = [LogisticRegression(random_state=r),
          RandomForestClassifier(random_state=r), MLPClassifier(random_state=r) ]

final_names = ["LogisticRegression","Random_Forest","Art.Neural_Network",]

In [59]:
# Possible hyper parameters
logreg_params= {"C":np.logspace(-1, 1, 10),
                    "penalty": ["l1","l2"], "solver":['lbfgs', 'liblinear', 'sag', 'saga'], "max_iter":[1000]}

rf_params = {"max_features": ["log2","auto","sqrt"],
                "min_samples_split":[2,3,5],
                "min_samples_leaf":[1,3,5],
                "bootstrap":[True,False],
                "n_estimators":[50,100,150],
                "criterion":["gini","entropy"]}

#nb_params = {'var_smoothing': np.logspace(0,-9, num=100)}


mlpc_params = {"alpha": [0.1, 0.01, 0.02, 0.005, 0.0001,0.00001],
              "hidden_layer_sizes": [(10,10,10),
                                     (100,100,100),
                                     (100,100),
                                     (3,5), 
                                     (5, 3)],
              "solver" : ["lbfgs","adam","sgd"],"max_iter":[1000]}


classifier_params = [logreg_params,rf_params, mlpc_params] 

In [ ]:
# Tuning by Cross Validation  
cv_result = {}
best_estimators = {}
for name, model,classifier_param in zip(final_names, final_models,classifier_params):
    with timer(">Model tuning"):
        clf = RandomizedSearchCV(model, classifier_param, cv=5, scoring = "roc_auc", n_jobs = -1,verbose = False)
        clf.fit(x_train,y_train)
        cv_result[name]=clf.best_score_
        best_estimators[name]=clf.best_estimator_
        print(name,'cross validation accuracy : %.4f'%cv_result[name])

### Validation Scores on Tuned Models

In [ ]:
roc_scores={}
print('Validation accuracies of the tuned models for the train data:', end = "\n\n")
for name, model_tuned in zip(best_estimators.keys(),best_estimators.values()):
    y_pred =  model_tuned.fit(x_train,y_train).predict_proba(x_val)[:,1]
    score=roc_auc_score(y_val, y_pred)
    print(name,':', "%.3f" %score)
    roc_scores[name]=score

We can see that Artfical Neural Network and Logistic Regression alone, gave high roc_auc_score, but to be certain we will rather go ahead with an ensemble model with best models to take care of the uncertainity in new data.

## Model Selection
Let's check the best model and then the ensemble models.

### Best Model

In [ ]:
n=1
scores=sorted(roc_scores, reverse=True, key= lambda k:roc_scores[k])[:n]
firstn=[[k,v] for k,v in best_estimators.items() if k in scores]

votingC = VotingClassifier(estimators = firstn, voting = "soft", n_jobs = -1)
votingC = votingC.fit(x_train, y_train)
pred1 = votingC.predict_proba(x_val)[:,1]  #Probabilities
print(roc_auc_score(y_val,pred1))

### Ensemble 2


In [ ]:
n=2
scores=sorted(roc_scores, reverse=True, key= lambda k:roc_scores[k])[:n]
firstn=[[k,v] for k,v in best_estimators.items() if k in scores]

# Ensembling First n Score
votingC = VotingClassifier(estimators = firstn, voting = "soft", n_jobs = -1)
votingC = votingC.fit(x_train, y_train)
pred2 = votingC.predict_proba(x_val)[:,1]  #Probabilities

print(roc_auc_score(y_val,pred2))

### Ensemble 3

In [ ]:
n=3
scores=sorted(roc_scores, reverse=True, key= lambda k:roc_scores[k])[:n]
firstn=[[k,v] for k,v in best_estimators.items() if k in scores]

# Ensembling First n Score
votingC = VotingClassifier(estimators = firstn, voting = "soft", n_jobs = -1)
votingC = votingC.fit(x_train, y_train)
pred2 = votingC.predict_proba(x_val)[:,1]  #Probabilities

print(roc_auc_score(y_val,pred2))

There are two major benefits of Ensemble models:

Better prediction
More stable model
The aggregate opinion of a multiple models is less noisy than other models. In finance, it is called “Diversification” a mixed portfolio of many stocks will be much less variable than just one of the stocks alone. This is also why our models will be better with ensemble of models rather than individual.

So my final model will be an Ensemble of .